In [9]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from social_lstm_classifier import SocialLSTMClassifier

trajectory_features = ['Positionx', 'Positiony', 'Distance', 'Speed', 'Speed Change', 'Direction', 'Direction Change']

feature_combos = {
    # 'xy_2': ['Positionx', 'Positiony'],
    # 'xyd_3': ['Positionx', 'Positiony', 'Distance'],
    # 'xydsdc_7': ['Positionx', 'Positiony', 'Distance', 'Speed', 'Direction', 'Speed Change', 'Direction Change'],
    # 'xysd_4': ['Positionx', 'Positiony', 'Speed', 'Direction'],
    # 'xyds_4': ['Positionx', 'Positiony', 'Distance', 'Speed'],
    # 'xydd_4': ['Positionx', 'Positiony', 'Distance', 'Direction'],
    # 'xydsc_5': ['Positionx', 'Positiony', 'Distance', 'Speed', 'Speed Change'],
    # 'xyddc_5': ['Positionx', 'Positiony', 'Distance', 'Direction', 'Direction Change'],
    # 'xys_3': ['Positionx', 'Positiony', 'Speed'],
    'xydi_3': ['Positionx', 'Positiony', 'Direction'],
    # 'xysc_4': ['Positionx', 'Positiony', 'Speed', 'Speed Change'],
    # 'xydc_4': ['Positionx', 'Positiony', 'Direction', 'Direction Change']
}

train_data = torch.load("./train_social_lstm_full.pt")
test_data = torch.load("./test_social_lstm_full.pt")

results = []

def run_experiment(feature_set, name):
    input_size = len(feature_set)
    index_map = [trajectory_features.index(f) for f in feature_set]

    model = SocialLSTMClassifier(input_size=input_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    epochs = 250

    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for sample in tqdm(train_data, desc=f"Epoch {epoch+1}/{epochs}"):
            traj = sample['trajectory'][:, index_map].unsqueeze(1)
            neighbors = sample['neighbors'][:, :, index_map]
            mask = sample['neighbor_mask']
            label = torch.tensor([int(sample['cluster']) - 1], dtype=torch.long)

            optimizer.zero_grad()
            logits = model(traj, neighbors, mask)
            loss = criterion(logits, label)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_data)
        print(f"\n[Feature Set: {name}] Epoch [{epoch+1}/{epochs}] - Avg Loss: {avg_loss:.4f}")


    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
       for sample in tqdm(train_data, desc=f"Epoch {epoch+1}/{epochs}"):
            traj = sample['trajectory'][:, index_map].unsqueeze(1)
            neighbors = sample['neighbors'][:, :, index_map]
            mask = sample['neighbor_mask']
            label = int(sample['cluster']) - 1
            logits = model(traj, neighbors, mask)
            pred = logits.argmax(dim=1).item()
            y_true.append(label)
            y_pred.append(pred)

    acc = accuracy_score(y_true, y_pred)
    clf_report = classification_report(y_true, y_pred, output_dict=True)
    results.append({
        "Feature_Set": name,
        "Accuracy": acc,
        "Macro_F1": clf_report['macro avg']['f1-score'],
        "Weighted_F1": clf_report['weighted avg']['f1-score']
    })

for name, feats in feature_combos.items():
    run_experiment(feats, name)



/var/folders/nt/7f1y5h8s6qd2mh1ypgjkgbpw0000gn/T/ipykernel_90241/428396086.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load("./train_social_lstm_


[Feature Set: xydi_3] Epoch [1/250] - Avg Loss: 0.6867


Epoch 2/250: 100%|██████████| 570/570 [00:03<00:00, 153.68it/s]



[Feature Set: xydi_3] Epoch [2/250] - Avg Loss: 0.6738


Epoch 3/250: 100%|██████████| 570/570 [00:03<00:00, 151.35it/s]


...

Epoch 249/250: 100%|██████████| 570/570 [00:03<00:00, 156.48it/s]



[Feature Set: xydi_3] Epoch [249/250] - Avg Loss: 0.2845


Epoch 250/250: 100%|██████████| 570/570 [00:03<00:00, 155.64it/s]



[Feature Set: xydi_3] Epoch [250/250] - Avg Loss: 0.2136


Epoch 250/250: 100%|██████████| 570/570 [00:01<00:00, 340.05it/s]


In [10]:
df_results = pd.DataFrame(results)
print(df_results)

  Feature_Set  Accuracy  Macro_F1  Weighted_F1
0      xydi_3   0.94386  0.938926     0.942885
